In [6]:
# 1) 모듈 임포트
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup

# 셀레니움 + 뷰티풀숩
- 참조 : Scrap_수업자료11. 

1) 셀레니움으로 크롬웹브라우저 실행, 웹페이지 이동
2) 현재 html 페이지를 변수에 저장
3) html 페이지 => 뷰티풀숩 객체로 변경
4) 뷰티풀숩의 메서드를 이용하여 정보 추출
5) 데이타프레임으로 변경 및 csv 저장


In [7]:
# 2) 셀레니움으로 크롬웹브라우저 실행, 웹페이지 이동

# 윈도우용
driver = webdriver.Chrome('chromedriver.exe')
time.sleep(2)
url = 'http://www.google.com'
driver.get(url)

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_16152\621673160.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [8]:
# 3) 원격 웹브라우저의 현재 html 페이지의 소스를 변수에 저장
# 드라이버 객체.page_source   >> 파이썬에 str로 변화

html_str = driver.page_source # requests에 res.text와 같은 과정
# html_str

In [11]:
# 4) html 페이지 => 뷰티풀숩 객체로 변경
soup = BeautifulSoup(html_str)  # html_parser
# soup

In [ ]:
# 5) 필요한 요소 추출 후 텍스트 출력
# 대한민국 글자만 표시하기
# 소스파악
# <div class="uU7dJb">대한민국</div>

In [17]:
soup.select_one('div.uU7dJb').text

'대한민국'

# 멜론 차트 순위
- https://www.melon.com/chart/

- 멜론 사이트를 Request 로 요청
- 406 Not Acceptable(접수할 수 없음)
- 에러코드 정리 : https://activeks.tistory.com/entry/ks-life

- 순위, 곡명, 아티스트, 앨범명, 앨범이미지, 앨범URL

In [21]:
import requests

In [33]:
url = "https://www.melon.com/chart/"
res = requests.get(url)
html_text = res.text

406


In [34]:
# 1) 셀레니움 드라이버 생성 후 멜론으로 이동

driver = webdriver.Chrome('chromedriver.exe')
time.sleep(2)

url = 'https://www.melon.com/chart/'
driver.get(url)

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_16152\1961190937.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [38]:
# 3) 원격 웹브라우저의 현재 html 페이지의 소스를 변수에 저장
# 드라이버 객체.page_source   >> 파이썬에 str로 변화

html_str = driver.page_source # requests에 res.text와 같은 과정

In [39]:
# 4) html 페이지 => 뷰티풀숩 객체로 변경
soup = BeautifulSoup(html_str)  # html_parser
# soup

# 5) 소스 파악

In [62]:
#<td><div class="wrap t_center"><span class="rank ">2</span><span class="none">위</span></div></td>

tbody > tr > td > div > span >'rank'

In [63]:
len(soup.select('td > div > span.rank'))

100

In [127]:
target = soup.select('tbody > tr')
len(target)

100

In [133]:
# 순위, 곡명, 아티스트, 앨범명, 앨범이미지, 앨범URL
base_url = 'https://www.melon.com/album/detail.htm?albumId='

rank = target[0].select_one('span.rank').text + '위'
song_name = target[0].select_one('div.ellipsis.rank01 span').text
singer = target[0].select_one('div.ellipsis.rank02 a').text
album = target[0].select_one('div.ellipsis.rank03 a').text
album_img = target[0].select_one('img')['src']
album_url = base_url + target[0].select_one('div a')['href'].split("'")[-2]
# print(target[0])

In [134]:
print(rank, song_name, singer, album, album_img, album_url)

1위 
That That (prod. & feat. SUGA of BTS)
 싸이 (PSY) 싸다9 https://cdnimg.melon.co.kr/cm2/album/images/109/37/474/10937474_20220428225312_500.jpg/melon/resize/120/quality/80/optimize https://www.melon.com/album/detail.htm?albumId=10937474


In [140]:
melon_100 = []

for i in target:
    rank = i.select_one('span.rank').text + ' 위'
    song_name = i.select_one('div.ellipsis.rank01 span').text.strip()
    singer = i.select_one('div.ellipsis.rank02 a').text
    album = i.select_one('div.ellipsis.rank03 a').text
    album_img = i.select_one('img')['src']
    album_url = base_url + i.select_one('div a')['href'].split("'")[-2]
    
    melon_100.append([rank, song_name, singer, album, album_img, album_url])
    
len(melon_100)

100

In [141]:
print(melon_100[0])

['1 위', 'That That (prod. & feat. SUGA of BTS)', '싸이 (PSY)', '싸다9', 'https://cdnimg.melon.co.kr/cm2/album/images/109/37/474/10937474_20220428225312_500.jpg/melon/resize/120/quality/80/optimize', 'https://www.melon.com/album/detail.htm?albumId=10937474']


In [142]:
df = pd.DataFrame(melon_100, columns=['순위', '곡명', '가수', '앨범', '앨범이미지', '앨범주소'])
df.to_csv('output/melon100.csv', index=False)
df

,순위,곡명,가수,앨범,앨범이미지,앨범주소
0,1 위,That That (prod. & feat. SUGA of BTS),싸이 (PSY),싸다9,https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
1,2 위,TOMBOY,(여자)아이들,I NEVER DIE,https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
2,3 위,LOVE DIVE,IVE (아이브),LOVE DIVE,https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
3,4 위,사랑인가 봐,멜로망스,사랑인가 봐 (사내맞선 OST 스페셜 트랙),https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
4,5 위,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life),https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
...,...,...,...,...,...,...
95,96 위,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL,https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
96,97 위,헤픈 우연,헤이즈 (Heize),HAPPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
97,98 위,초대,멜로망스,초대,https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...
98,99 위,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",AKMU (악뮤),항해,https://cdnimg.melon.co.kr/cm2/album/images/10...,https://www.melon.com/album/detail.htm?albumId...


# CGV

- 참조 소스 : Scrap_수업자료12
- http://section.cgv.co.kr/theater/timetable/Default.aspx
- 지역, 지점, url

1) 셀레니움으로 크롬웹브라우저 실행, CGV 웹페이지 이동
2) 지역탭 부분 클릭 -> 해당페이지로 이동(예시 - 부산)
3) 현재 html 페이지를 변수에 저장 > html text 페이지
4) html 페이지 > 뷰티풀숩 객체로 변경
5) 뷰티풀숩의 메서드를 이용하여 정보추출
6) 2~5번 과정을 반복 실행 한 후 2차원 리스트로 저장.
7) 데이터프레임으로 변경 및 csv 저장

# 부산 지역의 cgv 데이타만 저장

In [146]:
# 1) 셀레니움 드라이버 생성 후 멜론으로 이동

driver = webdriver.Chrome('chromedriver.exe')
time.sleep(2)

url = 'http://section.cgv.co.kr/theater/timetable/Default.aspx'
driver.get(url)

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_16152\75692533.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [151]:
# 2) 부산 탭 부분 클릭
# 부산탭 부분 요소 추출 
time.sleep(2)
target = driver.find_element_by_css_selector('a#ctl00_bodyPlaceHolder_rptRegion_ctl07_lbtnRegion')

# 요소 클릭
target.click()

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_16152\3370636119.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  target = driver.find_element_by_css_selector('a#ctl00_bodyPlaceHolder_rptRegion_ctl07_lbtnRegion')


In [152]:
# 3) 웹브라우저의 현재 html 페이지 소스를 저장 > bs로 변환
html_str = driver.page_source
soup = BeautifulSoup(html_str)

In [194]:
# 4) 정보 추출 
# 지역, 지점명, 지점url 
area = soup.select_one('a#ctl00_bodyPlaceHolder_rptRegion_ctl07_lbtnRegion').text
print(area)
# 지점 리스트 
csv_list = soup.select('div.tab_con_wrap_none > a')
print(len(csv_list)) 
# 첫번째 지점명과 하이퍼링크 
csv_name = csv_list[0].text

base = 'http://section.cgv.co.kr'
csv_url = base+csv_list[0]['href']


print(csv_name, csv_url)
csv_list[0]

부산
13
CGV 대연 http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0061


<a href="/theater/timetable/Default.aspx?code=0061" id="ctl00_bodyPlaceHolder_rptTheater_ctl00_aTheater"><span>CGV 대연</span></a>

In [196]:
# 5) 부산 전지역 CSV => 데이타프레임 
area = soup.select_one('a#ctl00_bodyPlaceHolder_rptRegion_ctl07_lbtnRegion').text

# 2차원 리스트
cgv_pusan = []
base_url = 'http://section.cgv.co.kr'
for cgv in csv_list:
    csv_name = cgv.text   
    csv_url = base+cgv['href']
    cgv_pusan.append([area, csv_name, csv_url])

df = pd.DataFrame(cgv_pusan, columns=['지역', '지점', 'URL'])
df

지역                                                    부산
지점                                            씨네드쉐프 센텀시티
URL    http://section.cgv.co.kr/theater/timetable/Def...
Name: 12, dtype: object

In [197]:
# 특정행의 정보 출력
df.loc[12]
df.loc[12, 'URL']

'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=P004'

# 모든 지점의 cgv
- 지역, 지점명, url

In [199]:
# 1) 셀레니움 드라이버 객체 생성 후 크롬 웹브라우저 구동 
# CGV 사이트로 이동 
driver = webdriver.Chrome('chromedriver.exe')
time.sleep(2)
url = 'http://section.cgv.co.kr/theater/timetable/Default.aspx'
driver.get(url)

C:\Users\pmpkc\AppData\Local\Temp\ipykernel_16152\3869181683.py:4: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  target = driver.find_elements_by_css_selector('a#C')


AttributeError: 'list' object has no attribute 'click'

# 2) 현재페이지를 저장하고 뷰티풀 숲으로 변환
html_str = driver.page_source
soup = BeautifulSoup(html_str)

## 소스파악
'li.theater_tab_area p a'

In [205]:
# 3) 지역텍스트와 탭메뉴부분의 id 값을 리스트로 변경
# 빈리스트 생성
area_list = [] # 지역명 리스트
area_id_list = [] #지역의 아이디값 리스트
target = soup.select('li.theater_tab_area p a')

# 각 리스트에 저장
for a in target:
    area_list.append(a.text)
    area_id_list.append(a['id'])
print(area_list)
print(area_id_list)


['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '인천', '전라', '제주', '충청']
['ctl00_bodyPlaceHolder_rptRegion_ctl00_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl01_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl02_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl03_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl04_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl05_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl06_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl07_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl08_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl09_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl10_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl11_lbtnRegion', 'ctl00_bodyPlaceHolder_rptRegion_ctl12_lbtnRegion']


In [215]:
# 4) 각지역탭(지역아이디)을 클릭해서 페이지 저장 
# 페이지를 숩객체로 변경
# 요소 추출 
# 요소 => 2차원 리스트로 저장 
# 빈리스트 저장 
result_list = []
base_url = 'http://section.cgv.co.kr'
idx = 0 
for id in area_id_list:
    print('지역 => ', area_list[idx], '아이디 => ', id )
    # 각 지역탭 클릭 
    area_id = 'a#'+id
    print(area_id)
    time.sleep(2)
    t = driver.find_element_by_css_selector(area_id)
    time.sleep(2)
    t.click()
    time.sleep(2)
    # 페이지 저장 
    html_str = driver.page_source
    soup = BeautifulSoup(html_str)
    # 요소 추출해서 리스트에 저장 
    csv_list = soup.select('div.tab_con_wrap_none > a')
    for cgv in csv_list:
        csv_name = cgv.text   
        csv_url = base + cgv['href']
        result_list.append([area_list[idx], csv_name, csv_url])
    idx += 1

지역 =>  강원 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl00_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl00_lbtnRegion


C:\Users\pmpkc\AppData\Local\Temp\ipykernel_16152\1579284279.py:15: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  t = driver.find_element_by_css_selector(area_id)


지역 =>  경기 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl01_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl01_lbtnRegion
지역 =>  경남 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl02_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl02_lbtnRegion
지역 =>  경북 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl03_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl03_lbtnRegion
지역 =>  광주 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl04_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl04_lbtnRegion
지역 =>  대구 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl05_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl05_lbtnRegion
지역 =>  대전 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl06_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl06_lbtnRegion
지역 =>  부산 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl07_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl07_lbtnRegion
지역 =>  서울 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl08_lbtnRegion
a#ctl00_bodyPlaceHolder_rptRegion_ctl08_lbtnRegion
지역 =>  인천 아이디 =>  ctl00_bodyPlaceHolder_rptRegion_ctl09_

In [219]:
result_list

[['강원',
  'CGV 강릉',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0139'],
 ['강원',
  'CGV 원주',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0144'],
 ['강원',
  'CGV 원통',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0354'],
 ['강원',
  'CGV 인제',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0281'],
 ['강원',
  'CGV 춘천',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0070'],
 ['경기',
  'CGV Drive In 곤지암',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0342'],
 ['경기',
  'CGV 경기광주',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0260'],
 ['경기',
  'CGV 고양행신',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0255'],
 ['경기',
  'CGV 광교',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0257'],
 ['경기',
  'CGV 광교상현',
  'http://section.cgv.co.kr/theater/timetable/Default.aspx?code=0266'],
 ['경기',
  'CGV 광명역',
  'http://section.cgv.co.kr/theater/timetab

In [220]:
# 5) 전국 전지역 CSV => 데이타프레임 
df = pd.DataFrame(result_list, columns=['지역', '지점', 'URL'])
df

,지역,지점,URL
0,강원,CGV 강릉,http://section.cgv.co.kr/theater/timetable/Def...
1,강원,CGV 원주,http://section.cgv.co.kr/theater/timetable/Def...
2,강원,CGV 원통,http://section.cgv.co.kr/theater/timetable/Def...
3,강원,CGV 인제,http://section.cgv.co.kr/theater/timetable/Def...
4,강원,CGV 춘천,http://section.cgv.co.kr/theater/timetable/Def...
...,...,...,...
154,충청,CGV 청주지웰시티,http://section.cgv.co.kr/theater/timetable/Def...
155,충청,CGV 청주터미널,http://section.cgv.co.kr/theater/timetable/Def...
156,충청,CGV 충북혁신,http://section.cgv.co.kr/theater/timetable/Def...
157,충청,CGV 충주교현,http://section.cgv.co.kr/theater/timetable/Def...


In [221]:
df.tail()

,지역,지점,URL
154,충청,CGV 청주지웰시티,http://section.cgv.co.kr/theater/timetable/Def...
155,충청,CGV 청주터미널,http://section.cgv.co.kr/theater/timetable/Def...
156,충청,CGV 충북혁신,http://section.cgv.co.kr/theater/timetable/Def...
157,충청,CGV 충주교현,http://section.cgv.co.kr/theater/timetable/Def...
158,충청,CGV 홍성,http://section.cgv.co.kr/theater/timetable/Def...
